In [1]:
import torch
import torch.nn as nn
from torchsummary import summary

In [2]:
class residual_block(nn.Module):
  def __init__(self,in_channel,out_channel,downsampling,stride):
    super(residual_block,self).__init__()
    self.conv1 = nn.Conv2d(in_channels=in_channel,out_channels=out_channel,kernel_size=3,stride=stride,padding=1)
    self.conv2 = nn.Conv2d(in_channels=out_channel,out_channels=out_channel,kernel_size=3,stride=1,padding=1)
    self.bnm = nn.BatchNorm2d(out_channel)
    self.relu = nn.ReLU(inplace=True)
    self.downsampling = downsampling
    self.conv3 = nn.Conv2d(in_channels=in_channel,out_channels=out_channel,kernel_size=3,stride=stride,padding=1)
  
  def forward(self,x):
    x_input = x
    x = self.conv1(x)
    x = self.bnm(x)
    x = self.relu(x)
    x = self.conv2(x)
    x = self.bnm(x)
    if self.downsampling == True:
      x_input = self.conv3(x_input)
    x = x + x_input
    x = self.relu(x)
    return x

In [3]:
class resnet(nn.Module):
  def __init__(self,num_class):
    super(resnet,self).__init__()
    self.conv = nn.Conv2d(in_channels=3,out_channels=64,kernel_size=7,stride=2,padding=3)
    self.maxpool = nn.MaxPool2d(kernel_size=3,stride=2,padding=1)
    self.flatten = nn.Flatten()
    self.avgpool = nn.AvgPool2d(kernel_size=7,stride=1,padding=0)
    self.bnm = nn.BatchNorm2d(64)
    self.relu = nn.ReLU(inplace=True)
    self.fc = nn.Linear(in_features=512,out_features=num_class)
    self.softmax = nn.Softmax(dim=1)
    self.block_1 = residual_block(64,64,False,1)
    self.block_2 = residual_block(64,128,True,2)
    self.block_3 = residual_block(128,128,False,1)
    self.block_4 = residual_block(128,256,True,2)
    self.block_5 = residual_block(256,256,False,1)
    self.block_6 = residual_block(256,512,True,2)
    self.block_7 = residual_block(512,512,False,1)

  def forward(self,x):
    x = self.conv(x)
    x = self.bnm(x)
    x = self.relu(x)
    x = self.maxpool(x)
    x = self.block_1(x)
    x = self.block_1(x)
    x = self.block_1(x)
    x = self.block_2(x)
    x = self.block_3(x)
    x = self.block_3(x)
    x = self.block_3(x)
    x = self.block_4(x)
    x = self.block_5(x)
    x = self.block_5(x)
    x = self.block_5(x)
    x = self.block_5(x)
    x = self.block_5(x)
    x = self.block_6(x)
    x = self.block_7(x)
    x = self.block_7(x)
    x = self.avgpool(x)
    x = self.flatten(x)
    x = self.fc(x)
    x = self.softmax(x)
    return x

In [4]:
model = resnet(1000)
summary(model,input_size=(3,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,472
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]          36,928
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,928
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
   residual_block-11           [-1, 64, 56, 56]               0
           Conv2d-12           [-1, 64, 56, 56]          36,928
      BatchNorm2d-13           [-1, 64, 56, 56]             128
             ReLU-14           [-1, 64,